In [ ]:
# Plan of Action
# plot all tackle distance data to determine methodology for picking playmaking threshold
    # Find distance between tackler and ball when ball carrier gets ball
# come up with cone of influence equation 
# distance delta pipeline


 #cone is where the defender could make the tackle

# ONLY LOOK AT RUN PLAYS FOR NOW
# Chase down efficiency
    # How well does each defender's cone move towards the ball carrier
        # Have a minimum distance to be able to be considered for the play
            # distribution of tackler's distance moved from play, get the upper quartile as the min distance
        # How do we judge how effectively they move there?
            # is it just distance delta?
                # consider derivative of distance over time as metric in concern
            # is it moving to final tackle location
            # is it cone overlap delta?

# Once cone overlap reaches threshold, chaser converts to prospective tackler
    # Use this to consider conversion rates for chasers

# Tackle probability - for all prospective tacklers
    # after ball carrier is in cone, how well does tackler make tackle
        # cone overlap integral
        # how fast does cone overlap area increase
        # 
##

In [41]:
import pandas as pd
import numpy as np
from datetime import datetime as dt

In [ ]:
# Game info - id, week, date, time, teams, scores
games = pd.read_csv("games.csv")


In [ ]:
plays = pd.read_csv("plays.csv")
plays['gameClockSeconds'] = pd.to_datetime(plays['gameClock'], format='%M:%S').dt.minute * 60 + pd.to_datetime(plays['gameClock'], format='%M:%S').dt.second
plays.sort_values(by=["gameId", "quarter", "gameClockSeconds"], ascending=[True, True, False], inplace=True)
# plays.head()
# print(plays[["gameId", "gameClockSeconds"]])
plays['penaltyYards'].fillna(0, inplace=True)
plays['passLength'].fillna(0, inplace=True)

plays['passResult'] = plays['passResult'].replace('R', 'Q')
plays['passResult'] = plays['passResult'].fillna('R')
print(plays['passResult'])
print(plays.columns)

In [ ]:
plays.drop("gameClock", axis=1, inplace=True)
plays.head()

In [ ]:
players = pd.read_csv("players.csv")
players['birthDate'] = pd.to_datetime(players['birthDate'])
reference_date = dt(2022, 9, 8)
players['age'] = players['birthDate'].apply(lambda x: reference_date.year - x.year - ((reference_date.month, reference_date.day) < (x.month, x.day)))
players.drop('birthDate', axis = 1, inplace=True)
players.drop('collegeName', axis = 1, inplace=True)
players.head()

In [ ]:
tackles = pd.read_csv("tackles.csv")
tackles_with_names = pd.merge(tackles, players[['nflId', 'displayName']], on='nflId', how='left')
# Drop the nflId column if it's no longer needed
#tackles_with_names.drop('nflId', axis=1, inplace=True)
tackles = tackles_with_names
tackles.head()

In [ ]:
w1 = pd.read_csv("tracking_week_1.csv")
w2 = pd.read_csv("tracking_week_2.csv")
w3 = pd.read_csv("tracking_week_3.csv")
w4 = pd.read_csv("tracking_week_4.csv")
w5 = pd.read_csv("tracking_week_5.csv")
w6 = pd.read_csv("tracking_week_6.csv")
w7 = pd.read_csv("tracking_week_7.csv")
w8 = pd.read_csv("tracking_week_8.csv")
w9 = pd.read_csv("tracking_week_9.csv")
all_weeks = pd.concat([w1, w2, w3, w4, w5, w6, w7, w8, w9])
all_weeks.head()
print(all_weeks.columns)

In [ ]:
all_weeks[all_weeks['displayName'] == "football"]

In [ ]:
# want ballcarrier data for each play
# player name, player id, gameid, playid, x, y, s, etc. for all the moments that they have recieved the ball from the qb
temp = w1
temp.dropna(subset=['event'], inplace=True)
# Assuming your DataFrame is named df and contains the 'event' column

# Filtering to get rows where 'event' column doesn't contain 'pass', 'tackle', or 'contact'
filtered_events = temp[~temp['event'].str.contains('pass|tackle|contact', na=False, case=False)]['event']

# Displaying the unique filtered events
unique_filtered_events = filtered_events.unique()
print(unique_filtered_events)

#temp = temp[~temp['event'].str.contains('tackle', na=False)]['event']
# temp = temp[~temp['event'].str.contains('contact', na=False)]['event']
# type(w1.iloc[1,16])
w1[w1['event'] == 'run']
plays[plays["playId"] == 101][plays["gameId"] == 2022090800]

In [ ]:
games.head()

In [51]:
plays.head()

,gameId,playId,ballCarrierId,ballCarrierDisplayName,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,...,preSnapVisitorTeamWinProbability,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPoints,expectedPointsAdded,foulName1,foulName2,foulNFLId1,foulNFLId2,gameClockSeconds
4889,2022090800,56,42489,Stefon Diggs,(15:00) (Shotgun) J.Allen pass short right to ...,1,1,10,BUF,LA,...,0.586653,-0.000031,0.000031,1.298699,0.004420,NaN,NaN,NaN,NaN,900
2819,2022090800,80,46076,Josh Allen,"(14:29) (No Huddle, Shotgun) J.Allen scrambles...",1,2,4,BUF,LA,...,0.586684,-0.013497,0.013497,1.303119,0.823571,NaN,NaN,NaN,NaN,869
7700,2022090800,101,47857,Devin Singletary,(13:54) D.Singletary right end to BUF 45 for 7...,1,1,10,BUF,LA,...,0.600181,-0.014850,0.014850,2.126690,0.562363,NaN,NaN,NaN,NaN,834
6265,2022090800,122,47857,Devin Singletary,(13:15) (Shotgun) J.Allen pass short right to ...,1,2,3,BUF,LA,...,0.615031,-0.019032,0.019032,2.689053,0.399209,NaN,NaN,NaN,NaN,795
3627,2022090800,146,47857,Devin Singletary,(12:33) (Shotgun) D.Singletary left tackle to ...,1,1,10,BUF,LA,...,0.634062,0.021616,-0.021616,3.088262,-0.514056,NaN,NaN,NaN,NaN,753


In [ ]:
all_weeks = pd.concat([w1, w2, w3, w4, w5, w6, w7, w8, w9])
team_changes = all_weeks.groupby('nflId')['club'].nunique()
players_changed_teams = list(team_changes[team_changes > 1].index)
changed_team_ids_int = [int(id) for id in players_changed_teams]
players_with_team_changes = players[players['nflId'].isin(changed_team_ids_int)]
player_names = players_with_team_changes['displayName']
print(player_names)

In [ ]:
def create_mixture_distribution(velocity, angle, fov=120):
    """
    Create a mixture of Gaussian distributions to model an asymmetric distribution that favors the player's direction of eyesight.
    """
    sigma = calculate_standard_deviation(velocity)
    angle_radians = np.radians(angle)

    # Primary Gaussian: In the direction of eyesight
    primary_rotation_matrix = np.array([[np.cos(angle_radians), -np.sin(angle_radians)], 
                                        [np.sin(angle_radians), np.cos(angle_radians)]])
    primary_covariance_matrix = np.dot(primary_rotation_matrix, np.dot(np.diag([sigma**2, (sigma/2)**2]), primary_rotation_matrix.T))
    primary_gaussian = multivariate_normal([0, 0], primary_covariance_matrix)

    # Secondary Gaussian: To cover the area opposite to the direction of eyesight (reduced probability)
    opposite_angle_radians = np.radians(angle - 180)  # Angle for the opposite direction
    secondary_rotation_matrix = np.array([[np.cos(opposite_angle_radians), -np.sin(opposite_angle_radians)], 
                                          [np.sin(opposite_angle_radians), np.cos(opposite_angle_radians)]])
    secondary_covariance_matrix = np.dot(secondary_rotation_matrix, np.dot(np.diag([(sigma/3)**2, (sigma/3)**2]), secondary_rotation_matrix.T))
    secondary_gaussian = multivariate_normal([0, 0], secondary_covariance_matrix)

    return primary_gaussian, secondary_gaussian

def plot_mixture_distribution(velocity, angle):
    """
    Plot the mixture of Gaussian distributions for the given velocity and angle.
    """
    x, y = np.mgrid[-10:10:.1, -10:10:.1]
    pos = np.dstack((x, y))
    primary_gaussian, secondary_gaussian = create_mixture_distribution(velocity, angle)
    
    # Combine the distributions
    combined_pdf = 400*primary_gaussian.pdf(pos) + 0.0 * secondary_gaussian.pdf(pos)  # Weighted mixture

    plt.contourf(x, y, combined_pdf)
    plt.title(f"Mixture Distribution for Player's Tackle Area\nVelocity: {velocity} m/s, Angle: {angle}°")
    plt.xlabel('X Position')
    plt.ylabel('Y Position')
    plt.gca().set_aspect('equal', adjustable='box')
    plt.show()

# Plotting the mixture distribution
plot_mixture_distribution(20, 30)


In [102]:
#calculate distance threshold
for index, play in tackles.iterrows(): 
    current_game_id = play['gameId']
    current_play_id = play['playId']
    tackler_id = play['nflId']
    ball_carrier_id = plays[(plays["gameId"] == current_game_id) & (plays["playId"] == current_play_id)]['ballCarrierId'].iloc[0]
    current_play = all_weeks[(all_weeks['gameId'] == current_game_id) & (all_weeks['playId'] == current_play_id)]
    
    #from current_play, filter out ballcarrier, football, tackler
    #then find where ballcarrier pos = football pos
    current_play = current_play[(current_play["nflId"] == tackler_id) | (current_play["nflId"] == ball_carrier_id) | (current_play["displayName"] == 'football')]
    frame = 0
    print("play:")
    for i in range(1,max(current_play['frameId'])):
        curr_frame = current_play[current_play['frameId'] == i]
        ball_carrier_x = curr_frame[curr_frame['nflId'] == ball_carrier_id]['x'].iloc[0]
        ball_carrier_y = curr_frame[curr_frame['nflId'] == ball_carrier_id]['y'].iloc[0]
        football_x = curr_frame[curr_frame['displayName'] == 'football']['x'].iloc[0]
        football_y = curr_frame[curr_frame['displayName'] == 'football']['y'].iloc[0]
        if (abs(ball_carrier_x - football_x) < 0.3) and (abs(ball_carrier_y - football_y) < 0.3):
            frame = i
            print((ball_carrier_x, ball_carrier_y), (football_x, football_y), "frame: " ,frame)
            break



        # if ((curr_frame[curr_frame['nflId'] == ball_carrier_id]['x'].iloc[0]) == (curr_frame[curr_frame['displayName'] == 'football']['x'].iloc[0])):
        #     print("yay")


# value = df[df['column_name'] == identifying_value]['target_column_name'].iloc[0]




play:
(78.34, 34.05) (78.1399993896484, 34.2700004577637) frame:  23
play:
(47.85, 24.12) (47.8499984741211, 23.8999996185303) frame:  24
play:
(63.89, 20.27) (63.9599990844727, 20.5300006866455) frame:  7
play:
(63.67, 14.64) (63.5900001525879, 14.8500003814697) frame:  8
play:
(83.23, 5.65) (83.1500015258789, 5.92000007629395) frame:  7
play:
(43.48, 1.73) (43.7799987792969, 1.47000002861023) frame:  15
play:
(44.35, 30.74) (44.1300010681152, 30.7299995422363) frame:  20
play:
(74.75, 35.49) (75.0100021362305, 35.4500007629395) frame:  5
play:
(97.07, 15.92) (96.8300018310547, 16.1200008392334) frame:  2
play:
(80.47, 29.91) (80.5100021362305, 29.6200008392334) frame:  19
play:
(88.02, 24.3) (88.129997253418, 24.0300006866455) frame:  18
play:
(77.58, 40.33) (77.6800003051758, 40.5699996948242) frame:  7
play:
(87.93, 19.85) (87.6699981689453, 19.6599998474121) frame:  23
play:
(45.44, 18.14) (45.4700012207031, 18.3600006103516) frame:  5
play:
(73.55, 24.01) (73.5299987792969, 24.03

KeyboardInterrupt: 